In [ ]:
import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt

# Correct the path to your image
img_path = r'C:\Users\91966\Desktop\Emotion_Detection\test\disgust\PrivateTest_3929526.jpg'

# Read the image
img = cv2.imread(img_path)
img = plt.imread(img_path)
plt.imshow(img, cmap='gray')

In [ ]:
import os

# now to count the number of images in train dataset of each category and similarly in test dataset of each category
train_dir='train'
test_dir='test'

def count_files_in_subdir(directory,index):
    counts={}
# now find all the subdirectories inside the above directory and count the number of images/files in each sub-directory

    for sub in os.listdir(directory):
        sub_dir_path=os.path.join(directory,sub)
        if os.path.isdir(sub_dir_path):
            counts[sub]=len(os.listdir(sub_dir_path))

# we get the dictionary now we will convert this dictionary into a pandas dataframe

    df=pd.DataFrame(counts,index=[index])
    return df

train_count = count_files_in_subdir(train_dir,'train')
print(train_count)

test_count = count_files_in_subdir(test_dir,'test')
print(test_count)

In [ ]:
train_count.transpose().plot(kind='bar')

In [ ]:
test_count.transpose().plot(kind='bar')
# we can see that we have class imbalance also so we should take care of it also

In [ ]:
# both train and test dataset have same distribution of points of each class

emotions=os.listdir(train_dir)
plt.figure(figsize=(25,20))

for i,emotion in enumerate(emotions,1):
    sub_dir=os.path.join(train_dir,emotion)
    img_path=os.path.join(sub_dir,os.listdir(sub_dir)[3])
    img=plt.imread(img_path)
    plt.subplot(3,4,i)
    plt.imshow(img,cmap='gray')
    plt.title(emotion,fontsize=40)

In [ ]:
img_width,img_height=48,48
batch_size=64
epochs=10
num_classes=7

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# it will apply random transformation to every image(if rescale parameter is given that will be applied to all images)
data_generator=ImageDataGenerator(rescale=1./255,
                                 validation_split=0.2)
# here currently we are not applying any transformation just normalizing the pixel values and splitting the data into validation dataset


train_generator=data_generator.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    subset='training')

validation_generator=data_generator.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    subset='validation')

test_generator=data_generator.flow_from_directory(
    test_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale')

In [ ]:
train_class_labels=train_generator.class_indices
print("Training class labels: ",train_class_labels)

In [ ]:
# now at start we will try to build a basic cnn model to predict the class label


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),kernel_initializer="glorot_uniform",
          padding="same",input_shape=(img_width,img_height,1)))
model.add(Activation('relu'))
model.add(Conv2D(64,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(Conv2D(256,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(512,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(Conv2D(512,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

# flattening and adding dense layers
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# output layer
model.add((Dense(num_classes)))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
# now we will use some callbacks

from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,CSVLogger,ModelCheckpoint

cnn_path = ''
name="Basic_CNN_Model.keras"
chk_path=os.path.join(cnn_path,name)

earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=6,
                             verbose=1,
                             min_delta=0.0001)

csv_logger = CSVLogger(os.path.join(cnn_path,'training.log'))

model_checkpoint = ModelCheckpoint(chk_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   verbose=1)

callbacks = [earlystop,reduce_lr,csv_logger,model_checkpoint]

In [ ]:
# train_steps_per_epoch = train_generator.samples//train_generator.batch_size + 1
# validation_steps_per_epoch = validation_generator.samples//validation_generator.batch_size + 1
# test_steps_per_epoch = test_generator.samples//test_generator.batch_size
# print(train_generator.samples,train_generator.batch_size,train_steps_per_epoch)
# print(validation_generator.samples,validation_generator.batch_size,validation_steps_per_epoch)
# print(test_generator.samples,test_generator.batch_size,test_steps_per_epoch)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history= model.fit(train_generator,
                   epochs=50,
                   validation_data=validation_generator,
                   callbacks=callbacks)

In [ ]:
def plot_training_history(history):
    acc=history.history['accuracy']
    val_acc=history.history['val_accuracy']
    loss=history.history['loss']
    val_loss=history.history['val_loss']

    epochs_range = range(len(acc))
    plt.figure(figsize=(20, 5))
    plt.subplot(1,2,1)
    plt.plot(epochs_range,acc,label='training Acuuracy')
    plt.plot(epochs_range,val_acc,label='validation Acuuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1,2,2)
    plt.plot(epochs_range,loss,label='training Loss')
    plt.plot(epochs_range,val_loss,label='validation Loss')
    plt.title('Training and Validation Loss')

    plt.show()

In [ ]:
plot_training_history(history)

In [ ]:
train_loss,train_accuracy =model.evaluate(train_generator)
test_loss,test_accuracy =model.evaluate(test_generator)
print("final train accuracy = {:.2f} , test accuracy = {:.2f}".format(train_accuracy*100, test_accuracy*100))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns

import numpy as np
true_classes = test_generator.classes
predicted_classes = np.argmax(model.predict(test_generator,steps=int(np.ceil(test_generator.samples/test_generator.batch_size))), axis=1)
class_labels = list(test_generator.class_indices.keys())
print(true_classes)
print(predicted_classes)
print(class_labels)

# generate the cinfusion matrix
cm = confusion_matrix(true_classes,predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
report=classification_report(true_classes,predicted_classes,target_names=class_labels)
print("classification Report : \n",report)

In [ ]:
# we can see that we are getting an accuracy of above 60 percent on test dataset but precision and recall
# for all the classes is very low and we can see that from confusion matrix that it is wrongly classifying many data points

In [ ]:
from keras.utils import plot_model

# making predictions for a random number of images
batch_size=test_generator.batch_size
print(batch_size)

# the Random_batch is the number of batch we are choosing for printing test images
Random_batch = np.random.randint(0,len(test_generator)-1)
print(len(test_generator),Random_batch)

# selecting 10 random imgaes indices
Random_img_index = np.random.randint(0,batch_size,10)
fig,axes = plt.subplots(nrows=2,ncols=5,figsize=(10,5),subplot_kw={'xticks': [], 'yticks': []})
for i,ax in enumerate(axes.flat):
    Random_img=test_generator[Random_batch][0][Random_img_index[i]]
    Random_img_label=np.argmax(test_generator[Random_batch][1][Random_img_index[i]],axis=0)
    Model_prediction=np.argmax(model.predict(tf.expand_dims(Random_img,axis=0),verbose=0),axis=1)[0]

    ax.imshow(Random_img.squeeze(),cmap='gray')
    color='green' if class_labels[Random_img_label] == class_labels[Model_prediction] else "red"
    ax.set_title(f"True: {class_labels[Random_img_label]}\nPredicted: {class_labels[Model_prediction]}", color=color)


plt.show()

In [ ]:
# as we can see through the graphs that our model is overfitting to the training data and hence we can use data transformation techniques
# so that each time our model sees differently transformed version of the image and hence there can be a chance that it doesn't overfit

**Same CNN model with image augmentation**


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# it will apply random transformation to every image(if rescale parameter is given that will be applied to all images)
data_generator=ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest',
                                  validation_split=0.2)
# here currently we are not applying any transformation just normalizing the pixel values and splitting the data into validation dataset


train_generator=data_generator.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    subset='training')

validation_generator=data_generator.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    subset='validation')

test_generator=data_generator.flow_from_directory(
    test_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
image_path = 'C:\Users\91966\Desktop\Emotion_Detection\test\disgust\PrivateTest_807646.jpg'
img=load_img(image_path,color_mode='grayscale',target_size=(img_width,img_height))
img_array=img_to_array(img)
img_array=img_array.reshape((1,)+img_array.shape)

fig,axes = plt.subplots(nrows=1,ncols=5,figsize=(10,4))

axes[0].imshow(img_array[0,:,:,0],cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

for i,ax in enumerate(axes.flat[1:]):
    aug_iter = data_generator.flow(img_array,batch_size=1)
    aug_img = next(aug_iter)[0]
    ax.imshow(aug_img[:,:,0],cmap='gray')
    ax.set_title(f'Augmented Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# now we will use some callbacks

from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

cnn_path = ''
name="Data_Augmentation.keras"
chk_path=os.path.join(cnn_path,name)

earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=6,
                             verbose=1,
                             min_delta=0.0001)

csv_logger = CSVLogger(os.path.join(cnn_path,'training_with_Data_Augmentation2.log'))

model_checkpoint = ModelCheckpoint(chk_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   verbose=1)

callbacks = [earlystop,reduce_lr,csv_logger,model_checkpoint]

In [ ]:
# we can see that we are still not getting better recall and precision values
# we can use the class weights metric to check whether we are getting better result with that or not

from sklearn.utils.class_weight import compute_class_weight

classes = np.array(train_generator.classes)
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(classes),
                                     y=classes)

class_weights_dict = dict(enumerate(class_weights))
print("Class Weights Dictionary:", class_weights_dict)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),kernel_initializer="glorot_uniform",
          padding="same",input_shape=(img_width,img_height,1)))
model.add(Activation('relu'))
model.add(Conv2D(64,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(Conv2D(256,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(512,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(Conv2D(512,kernel_size=(3,3),padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

# flattening and adding dense layers
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# output layer
model.add((Dense(num_classes)))
model.add(Activation('softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history= model.fit(train_generator,
                   epochs=50,
                   validation_data=validation_generator,
                   class_weight=class_weights_dict,
                   callbacks=callbacks)

In [ ]:
from tensorflow.keras.models import load_model

# Load the model with the best weights
best_model = load_model(chk_path)


In [ ]:
plot_training_history(history)

In [ ]:
train_loss, train_accu = best_model.evaluate(train_generator)
test_loss, test_accu = best_model.evaluate(test_generator)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

In [ ]:
# Assuming your true_classes and predicted_classes are already defined
true_classes = test_generator.classes
predicted_classes = np.argmax(best_model.predict(test_generator, steps=int(np.ceil(test_generator.samples/test_generator.batch_size))), axis=1)
class_labels = list(test_generator.class_indices.keys())

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plotting with seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# Printing the classification report
report = classification_report(true_classes,
                               predicted_classes,
                               target_names=class_labels,
                               zero_division=0)
print("Classification Report:\n", report)

**The Below Code is to actually add new images to the current dataset and make number of elements in all class equal.**

In [ ]:
# import shutil
# from tensorflow.keras.preprocessing.image import save_img
# from tensorflow.keras.preprocessing.image import load_img, img_to_array

# dataset_dir='/kaggle/input/fer2013/train'
# augmented_dataset_dir='/kaggle/working/'

# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# class_counts={}
# for class_name in os.listdir(dataset_dir):
#     class_dir = os.path.join(dataset_dir,class_name)
#     if os.path.isdir(class_dir):
#         class_counts[class_name] = len(os.listdir(class_dir))
# print(class_counts)
# max_count=max(class_counts.values())

# def augment_and_save_images(image_path,save_dir,augmentation_needed):
#     img=load_img(image_path)
#     x=img_to_array(img)
#     x=np.expand_dims(x,axis=0)

#     i=0
#     for batch in datagen.flow(x,batch_size=1,save_to_dir=save_dir,save_prefix='aug',save_format='jpg'):
#         i+=1
#         if i>=augmentations_needed:
#             break
# print(max_count)
# for class_name,count in class_counts.items():
#     if count<max_count:
#         class_dir=os.path.join(dataset_dir,class_name)
#         print(class_dir)
#         save_dir = os.path.join(augmented_dataset_dir,class_name)
#         os.makedirs(save_dir,exist_ok=True)
#         print(save_dir)
#         augmentations_needed=max_count-count

#         images = os.listdir(class_dir)
#         augmentations_per_image = augmentations_needed // count
#         remaining_augmentations = augmentations_needed % count
#         print(augmentations_per_image,remaining_augmentations,count)
#         for image_name in images:
#             image_path = os.path.join(class_dir,image_name)
#             print(image_path)
#             augment_and_save_images(image_path,save_dir,augmentations_per_image)

# #         selected_images = np.random.choice(images,remaining_augmentations,replace=False)
# #         for image_name in selected_images:
# #             image_path = os.path.join(class_dir,image_name)
# #             augment_and_save_images(image_path,save_dir,1)

# print("data Augmentaion complete.The dataset is balanced now.")

# # copying original images to the augmented directory
# for class_name in os.listdir(original_dataset_dir):
#     class_dir=os.path.join(dataset_dir,class_name)
#     augmented_class_dir = os.path.join(augmented_dataset_dir,class_name)
#     os.makedirs(augmented_class_dir,exist_ok=True)
#     for filename in os.listdir(original_class_dir):
#         src=os.path.join(original_class_dir, filename)
#         dst= os.path.join(augmented_class_dir,filename)
#         shutil.copyfile(src,dst)

# print("Original images copied to augmented directory successfully.")


In [ ]:
# we can still see that the model acuracy didn't increase much so we can try with actually augmenting
# the minority class images so that all classes have equal number of images and wwe will be using the same cnn model

In [ ]:
img_width, img_height = 224, 224  # Size of images
batch_size = 64
epochs = 10
num_classes = 7

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# it will apply random transformation to every image(if rescale parameter is given that will be applied to all images)
data_generator=ImageDataGenerator(rescale=1./255,
                                 validation_split=0.2)
# here currently we are not applying any transformation just normalizing the pixel values and splitting the data into validation dataset


train_generator=data_generator.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    subset='training')

validation_generator=data_generator.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    subset='validation')

test_generator=data_generator.flow_from_directory(
    test_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50V2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D

tf.keras.backend.clear_session()

# load the vgg16 model, excluding the fully connected layers
vgg = VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')
vgg.summary()

In [ ]:
for layer in vgg.layers[:-3]:
  layer.trainable = False
vgg.summary()

In [ ]:
from tensorflow.keras.models import Sequential, Model

x=Flatten()(vgg.output)

x=Dense(1024,activation='relu',kernel_initializer='he_normal')(x)
x=Dropout(0.5)(x)

x=Dense(512,activation='relu',kernel_initializer='he_normal')(x)
x=Dropout(0.5)(x)

output = Dense(7,activation='softmax',kernel_initializer='he_normal')(x)

model=Model(inputs=vgg.input,outputs=output)

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False),
              metrics=['accuracy'])

model.summary()

In [ ]:
# now we will use some callbacks

from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,CSVLogger,ModelCheckpoint

cnn_path = ''
name="VGG16_Transfer_Learning.keras"
chk_path=os.path.join(cnn_path,name)

earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=6,
                             verbose=1,
                             min_delta=0.0001)

csv_logger = CSVLogger(os.path.join(cnn_path,'training_with_vggNet.log'))

model_checkpoint = ModelCheckpoint(chk_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   verbose=1)

callbacks = [earlystop,reduce_lr,csv_logger,model_checkpoint]

In [ ]:
history = model.fit(
                    train_generator,
                    epochs=50,
                    validation_data=validation_generator,
                    class_weight=class_weights_dict,
                    callbacks = callbacks
                    )

In [ ]:
plot_training_history(history)

train_loss, train_accu = model.evaluate(train_generator)
test_loss, test_accu = model.evaluate(test_generator)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns

import numpy as np
true_classes = test_generator.classes
predicted_classes = np.argmax(model.predict(test_generator,steps=int(np.ceil(test_generator.samples/test_generator.batch_size))), axis=1)
class_labels = list(test_generator.class_indices.keys())
print(true_classes)
print(predicted_classes)
print(class_labels)

# generate the cinfusion matrix
cm = confusion_matrix(true_classes,predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Printing the classification report
report = classification_report(true_classes,
                               predicted_classes,
                               target_names=class_labels,
                               zero_division=0)
print("Classification Report:\n", report)

In [ ]:
# still precision and recall didn't increase significantly

**RESNet 50**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255,
                                   rotation_range=10,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    class_mode='categorical',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    shuffle=True,
                                                    batch_size=batch_size
)

test_generator = test_datagen.flow_from_directory(
                                                  test_dir,
                                                  class_mode="categorical",
                                                  target_size=(224, 224),
                                                  color_mode="rgb",
                                                  shuffle=False,
                                                  batch_size=batch_size
                                                 )

In [ ]:
ResNet50V2 = tf.keras.applications.ResNet50V2(input_shape=(224,224,3),
                                              include_top=False,
                                              weights='imagenet')

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model=Sequential([ResNet50V2,
                  Dropout(0.25),
                  BatchNormalization(),
                  Flatten(),
                  Dense(64,activation='relu'),
                  BatchNormalization(),
                  Dropout(0.5),
                  Dense(7,activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,CSVLogger,ModelCheckpoint

cnn_path = ''
name='ResNet50_Transfer_Learning.keras'
chk_path=os.path.join(cnn_path,name)

checkpoint =ModelCheckpoint(filepath=chk_path,
                            save_best_only=True,
                            verbose=1,
                            mode='min',
                            monitor='val_loss'
)

earlystop = EarlyStopping(monitor='val_accuracy',
                          patience=7,
                          restore_best_weights=True,
                          verbose=1)

reduce_lr = ReduceLROnPlateau(monnitor='val_loss',
                              factor=0.2,
                              patience=2,
                              verbose=1)

csv_logger = CSVLogger(os.path.join(cnn_path,'training.log'))

callbacks = [checkpoint, earlystop, reduce_lr,csv_logger]

In [ ]:
train_history = model.fit(train_generator,
                          epochs=50,
                          validation_data=test_generator,
                          class_weight=class_weights_dict,
                          callbacks=callbacks)

In [ ]:
plot_training_history(train_history)

In [ ]:
train_loss, train_accu = model.evaluate(train_generator)
test_loss, test_accu = model.evaluate(test_generator)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

In [ ]:
true_classes=test_generator.classes
predicted_classes=np.argmax(model.predict(test_generator,steps=int(np.ceil
                            (test_generator.samples/test_generator.batch_size))),axis=1)
class_labels=list(test_generator.class_indices.keys())

cm=confusion_matrix(true_classes,predicted_classes)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
report = classification_report(true_classes,
                               predicted_classes,
                               target_names=class_labels,
                               zero_division=0)
print("Classification Report:\n", report)

**AUC-ROC Curve**

In [ ]:
y_encoded = pd.get_dummies(true_classes).astype(int).values
preds_encoded = pd.get_dummies(predicted_classes).astype(int).values

fpr=dict()
tpr=dict()
roc_auc=dict()
for i in range(7):
  fpr[i],tpr[i],_=roc_curve(y_encoded[:i],preds_encoded[:,i])
  roc_auc[i]=auc(fpr[i],tpr[i])

plt.figure(figsize=(10,5))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
for i, color in enumerate(colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f"ROC curve for {classes[i]} (area = {roc_auc[i]:0.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')

In [ ]:
model.save("Resnet_model_version_2.keras")